## 유튜브 스크립트 수집
youtube-transcript-api 라이브러리를 사용하여 영상의 스크립트를 수집할 수 있습니다.
이 코드에선 스크립트를 수집하는 방법을 보여줍니다.

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import re
from tqdm import tqdm


In [8]:
# 데이터 불러오기
df = pd.read_csv("../data/video_sample.csv", encoding="utf-8-sig", index_col=0)
df.head()

,video_id,channel_id,video_title,video_description,video_tags,video_duration,video_published,video_category,video_info_card,video_with_ads,video_end_screen,video_cluster,crawled_date,year,month,day
0,JCqgRM-2_GU,UCU-wl-PZYxpWeDl1OIiE_hQ,어머니와 함께 방문 한 15 세 중학생 환자분의 포경수술 영상 (포경 소매법) C...,수술 적응증 상담 : 010-9190-7575\n카카오톡 상담 : gnjurolog...,"['강남비뇨기과', '박천진', '포경', '포경수술', '귀두포피', '위생관리']",75,2024-10-06,Education,0,0,0,-1,2024-10-07 17:37:39,2024,10,6
1,R0pRp-3S5Yo,UCm69xuysxVgNuRpmXCiXlPw,YURIAN (유리안) - FUCKING MY CITY 2024 ver. (Ft. ...,#YURIAN #유리안 #FUCKING_MY_CITY_2024 #Rredrain #...,[],177,2024-10-06,Music,0,0,0,-1,2024-10-07 17:37:57,2024,10,6
2,RsVKZrt2DSo,UCVcLszfUBN9rTmc0A0uDRSQ,대리모 사건이 소환한 합법화 논쟁 / KBC뉴스,불임이나 난임 부부가 브로커에게 돈을 주고 출산을 의뢰한 '대리모 범죄'가 14년 ...,"['kbc', 'kbc광주방송', '깨비씨', 'kbc 뉴스']",158,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:38:23,2024,10,6
3,2CpgC1K7jNI,UCxCsxXsJPDMfjcXEEV09m9w,대포죽순이요????푸바오 핸펀몰카사건? 10.6 푸바오 이야기 FUBAO PANDA...,NaN,[],400,2024-10-06,Travel & Events,0,0,0,-1,2024-10-07 17:38:24,2024,10,6
4,C-jOF1sgdOg,UCYCb_mX7P1xKrB5iaT8y0Wg,"일본에 남겨진 한국의 문화유산【아시아의 숨은 혼, 백제를 가다 Part 2】",0:00 2012년 6월 18일 다카이다야마 고분\n1:43 2012년 6월 25일...,"['TJB', 'tjb', 'tjb대전방송', '엑스포로131', '엑스포로', '...",572,2024-10-07,Entertainment,0,0,0,-1,2024-10-07 17:38:24,2024,10,7


In [4]:
def collect_script(video_id):
    try:
        # 한국어 자막 요청 (자동 생성된 자막)
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])

        script = str()  # 스크립트를 저장할 빈 문자열 생성
        
        # 자막을 출력
        for entry in transcript:
            # 자막에서 대괄호 안의 내용을 제거하고 양쪽 공백 제거
            cleaned_text = re.sub(r'\[.*?\]', '', entry['text']).strip()
            # 자막이 "다"로 끝나면 마침표를 추가
            if cleaned_text.endswith("다"):
                cleaned_text += ". "
            else:
                cleaned_text += " "  # 다른 경우는 공백 추가
            
            # cleaned_text가 비어있지 않은 경우 스크립트에 추가
            if cleaned_text != "":
                script += cleaned_text
                
        return True, script  # 스크립트 수집 성공 시 True와 스크립트 반환
    except Exception as e:
        return False, str(e)  # 예외 발생 시 False와 에러 메시지 반환


In [10]:
scripts = pd.DataFrame()
scripts["video_id"] = df.video_id
scripts["video_title"] = df.video_title
scripts["video_script"] = None

# 데이터프레임의 각 행을 순회하면서 스크립트를 수집
for row in tqdm(df.iterrows(), total=len(df)):
    video_id = row[1].video_id  # 현재 행의 비디오 ID 저장
    # collect_script 함수를 호출하여 비디오 스크립트 수집
    result, script = collect_script(video_id)
    
    # 스크립트 수집이 성공한 경우
    if result:
        scripts.at[row[0], "video_script"] = script


100%|██████████| 101/101 [02:26<00:00,  1.45s/it]


In [14]:
# 수집된 스크립트만 저장
scripts = scripts[~scripts.video_script.isnull()].reset_index(drop=True)
scripts.to_csv("../data/video_script.csv", encoding="utf-8-sig")